In [1]:
# Import necessary packages

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import torch

#import helper

import matplotlib.pyplot as plt

In [2]:
import torchvision
from torch import nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch import optim

In [3]:
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [4]:
data_dir = 'Cat_Dog_data'

# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

In [5]:
def plot_img(image):
    image = image.numpy()[0]
    mean = 0.5
    std = 0.5
    image = ((mean*image)+std)
    plt.imshow(image,cmap='gray')

In [17]:
class Net (nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d (3, 16, 5, stride = 2, padding =1)
        self.conv2 = nn.Conv2d (16, 32, 5, stride =2 ,padding =1)
        self.conv3 = nn.Conv2d(32, 64, 3, stride = 1, padding = 1)
        self.pool = nn.MaxPool2d(2,2)
        #dimensions of the fc1 input layer = 64*6*6 (depth * reduced layer dimensions)
        self.fc1 = nn.Linear((64*6*6), 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 2)
        self.dropout = nn.Dropout (0.2)
        
    def forward (self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        
        x = x.view (x.shape[0], -1)
        x = self.dropout(x)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = F.log_softmax(self.fc3(x), dim =1)
        
        return x

model = Net()
print (model)

if train_on_gpu:
    model.cuda()

Net(
  (conv1): Conv2d(3, 16, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=2304, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=2, bias=True)
  (dropout): Dropout(p=0.2)
)


In [18]:

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [19]:
epochs = 15
steps = 0

train_losses, test_losses = [], []
for e in range(epochs):
    accuracyt = 0
    running_loss = 0
    for images, labels in trainloader:
        
        if train_on_gpu:
            images, labels = images.cuda(), labels.cuda()
        optimizer.zero_grad()
        
        log_ps = model(images)
        ps = torch.exp(log_ps)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracyt += torch.mean(equals.type(torch.FloatTensor))
        
        loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    else:
        test_loss = 0
        accuracy = 0
        
        # Turn off gradients for validation, saves memory and computations
        with torch.no_grad():
            model.eval()
            for images, labels in testloader:
                if train_on_gpu:
                    images, labels = images.cuda(), labels.cuda()
                log_ps = model(images)
                test_loss += criterion(log_ps, labels)
                
                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
        
        model.train()
        
        train_losses.append(running_loss/len(trainloader))
        test_losses.append(test_loss/len(testloader))

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(train_losses[-1]),
              "Training Accuracy: {:.3f}".format(accuracyt/len(trainloader)),
              "Test Loss: {:.3f}.. ".format(test_losses[-1]),
              "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))

Epoch: 1/15..  Training Loss: 0.654..  Training Accuracy: 0.610 Test Loss: 0.599..  Test Accuracy: 0.664
Epoch: 2/15..  Training Loss: 0.608..  Training Accuracy: 0.672 Test Loss: 0.630..  Test Accuracy: 0.648
Epoch: 3/15..  Training Loss: 0.579..  Training Accuracy: 0.693 Test Loss: 0.510..  Test Accuracy: 0.754
Epoch: 4/15..  Training Loss: 0.559..  Training Accuracy: 0.711 Test Loss: 0.483..  Test Accuracy: 0.771
Epoch: 5/15..  Training Loss: 0.537..  Training Accuracy: 0.732 Test Loss: 0.460..  Test Accuracy: 0.797
Epoch: 6/15..  Training Loss: 0.527..  Training Accuracy: 0.738 Test Loss: 0.451..  Test Accuracy: 0.791
Epoch: 7/15..  Training Loss: 0.513..  Training Accuracy: 0.749 Test Loss: 0.448..  Test Accuracy: 0.801
Epoch: 8/15..  Training Loss: 0.498..  Training Accuracy: 0.755 Test Loss: 0.434..  Test Accuracy: 0.804
Epoch: 9/15..  Training Loss: 0.494..  Training Accuracy: 0.762 Test Loss: 0.388..  Test Accuracy: 0.829
Epoch: 10/15..  Training Loss: 0.484..  Training Accura

In [20]:
torch.save(model.state_dict(), 'model2 cnn85accuracy.pth')